### Loading the dataset

In [ ]:
! pip install -U datasets

In [1]:
from datasets import load_dataset

dataset = load_dataset("wikimedia/wikipedia", "20231101.en")

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

In [2]:
 dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 6407814
    })
})

In [3]:
articles = dataset["train"][:1000]["text"]

In [4]:
len(articles)

1000

In [5]:
articles = [x.split("\n")[0] for x in articles]

In [6]:
articles[0]

'Anarchism is a political philosophy and movement that is skeptical of all justifications for authority and seeks to abolish the institutions it claims maintain unnecessary coercion and hierarchy, typically including nation-states, and capitalism. Anarchism advocates for the replacement of the state with stateless societies and voluntary free associations. As a historically left-wing movement, this reading of anarchism is placed on the farthest left of the political spectrum, usually described as the libertarian wing of the socialist movement (libertarian socialism).'

In [7]:
import tiktoken

In [8]:
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

In [9]:
from tqdm import tqdm
total_tokens = 0
for article in articles:
    total_tokens += len(encoding.encode(article))

In [10]:
total_tokens

88327

In [11]:
(total_tokens * 0.5 ) / 1000000

0.0441635

In [12]:
categories = ["History",
"Science",
"Technology",
"Arts",
"Literature",
"Geography",
"Politics",
"Philosophy",
"Religion",
"Mathematics",
"Medicine",
"Culture",
"Sports",
"Economics",
"Education",
"Environment",
"Society",
"Law",
"Language",
"Others"]

In [13]:
from getpass import getpass

OPENAI_API_KEY = getpass()

········


In [14]:
import os

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [15]:
from langchain_openai import OpenAI

llm = OpenAI()

In [16]:
llm.invoke("Tell me a joke")

"\n\nWhy couldn't the bicycle stand up by itself?\n\nBecause it was two-tired!"

In [60]:
prompt = f"""Based on the following article content, identify the category of the article

Article: {article}
Categories: {categories}

If the article doesn't fit into any of the above mentioned categories. Please return the answer as "Others"
Return only one category name
Answer:
"""

In [61]:
llm.invoke(prompt)

'\nPhilosophy'

In [17]:
answers = []

for article in tqdm(articles[:100]):
    prompt = f"""Based on the following article content, identify the category of the article

Article: {article}
Categories: {categories}

If the article doesn't fit into any of the above mentioned categories. Please return the answer as "Others"
Return only one category name
Answer:
"""
    answers.append(llm.invoke(prompt))



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:43<00:00,  2.27it/s]


In [19]:
import pandas as pd

df = pd.DataFrame({
    "Article": articles[:100],
    "Category": answers
})

In [27]:
df.loc[4]

Article     In Greek mythology, Achilles ( ) or Achilleus ...
Category                                           Literature
Name: 4, dtype: object

In [28]:
(7000000000 * 0.5) / 1000000

3500.0